# Start a Simulated SNMP Agent

In [1]:
import subprocess

try:
    if process:
        pass
except:
    process = subprocess.Popen(
        [
            'snmpsimd.py',
            '--agent-udpv4-endpoint=127.0.0.1:1161',
            '--agent-udpv6-endpoint=[::1]:1161'
        ],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
        shell=False
    )

# Create a DataFrame Mimicking a Database Query Result

In [2]:
from uuid import UUID

import pandas as pd

df = (
    pd.DataFrame(
        data=[
            [UUID('3a8083c9-79ff-4ed9-969c-e204cee391b3'), 'host1', 'localhost:1161', 'recorded/linux-full-walk', 'DNS Resolution'],
            [UUID('3cae774a-ec98-4679-b20a-dc1947b0546f'), 'host3', '[::1]:1161', 'recorded/linux-full-walk', 'IPv6'],
            [UUID('3cae774a-ec98-4679-b20a-dc1947b0546f'), 'host3', '[::1]:1161', 'recorded/linux-full-walk', 'Preserved Index'],
        ],
        columns=['id', 'hostname', 'ip_address', 'community_string', 'notes']
    )
    .set_index(['id', 'hostname'])
)
df

ip_address  \
id                                   hostname                   
3a8083c9-79ff-4ed9-969c-e204cee391b3 host1     localhost:1161   
3cae774a-ec98-4679-b20a-dc1947b0546f host3         [::1]:1161   
                                     host3         [::1]:1161   

                                                       community_string  \
id                                   hostname                             
3a8083c9-79ff-4ed9-969c-e204cee391b3 host1     recorded/linux-full-walk   
3cae774a-ec98-4679-b20a-dc1947b0546f host3     recorded/linux-full-walk   
                                     host3     recorded/linux-full-walk   

                                                         notes  
id                                   hostname                   
3a8083c9-79ff-4ed9-969c-e204cee391b3 host1      DNS Resolution  
3cae774a-ec98-4679-b20a-dc1947b0546f host3                IPv6  
                                     host3     Preserved Index

# Construct a MIB-like Definition of the SNMP Objects to Collect

In [3]:
import numpy as np

from snmp_fetch import ObjectType, object_type, pipeline_hook

@object_type(oid='.1.3.6.1.2.1.1.3.0')
class Uptime(ObjectType):
    """Uptime example."""

    dtype=np.dtype([('uptime', np.uint64)])  # shared index collected after the full OID has been constructed
    
    @pipeline_hook('before_pivot')
    def set_type(df):
        df['uptime'] = df['uptime'].inet.to_timedelta(100)
        return df

In [4]:
from snmp_fetch import PduType, SnmpConfig, fetch

results, errors = fetch(
    PduType.GET,
    df,
    Uptime,
    config=SnmpConfig(retries=0, timeout=1),
    host='ip_address',
    snmp_community='community_string'
)

errors # display errors

[]

In [5]:
results # display results

#timestamp  \
id                                   hostname                             
3a8083c9-79ff-4ed9-969c-e204cee391b3 host1    2019-11-18 18:37:37+00:00   
3cae774a-ec98-4679-b20a-dc1947b0546f host3    2019-11-18 18:37:37+00:00   
                                     host3    2019-11-18 18:37:37+00:00   

                                                        uptime  \
id                                   hostname                    
3a8083c9-79ff-4ed9-969c-e204cee391b3 host1    27 days 00:24:11   
3cae774a-ec98-4679-b20a-dc1947b0546f host3    27 days 00:24:11   
                                     host3    27 days 00:24:11   

                                                   ip_address  \
id                                   hostname                   
3a8083c9-79ff-4ed9-969c-e204cee391b3 host1     localhost:1161   
3cae774a-ec98-4679-b20a-dc1947b0546f host3         [::1]:1161   
                                     host3         [::1]:1161   

                                                       community_string  \
id                                   hostname                             
3a8083c9-79ff-4ed9-969c-e204cee391b3 host1     recorded/linux-full-walk   
3cae774a-ec98-4679-b20a-dc1947b0546f host3     recorded/linux-full-walk   
                                     host3     recorded/linux-full-walk   

                                                         notes  
id                                   hostname                   
3a8083c9-79ff-4ed9-969c-e204cee391b3 host1      DNS Resolution  
3cae774a-ec98-4679-b20a-dc1947b0546f host3                IPv6  
                                     host3     Preserved Index

# Stop the Simulated SNMP Agent

In [6]:
process.kill()
process.communicate()
del process